# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle
import random
from random import sample

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import Conditional_VAE as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_file = config['IO_OPTION']['OUTPUT_ROOT']+'/train_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_file, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(777)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
#dev_dir = INPUT_ROOT + "/dev_data"
#add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = ['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/CenterLossVAE/ex15_subseq/config.yaml'

In [7]:
torch.cuda.is_available()

True

In [8]:
#ext_data = pd.read_pickle(f'{INPUT_ROOT}/ToyCar_valid_source_features.pkl')

In [9]:
#ext_data['features'][-1].shape

In [10]:
# 938(subseq length) * 600(n_samples)
# wav_name = []
# for i in range(len(ext_data['wav_name'])):
#     per_wav_name = [ext_data['wav_name'][i]]
#     per_n_samples = ext_data['features'][0].shape[0]
#     wav_name.extend(per_wav_name * per_n_samples)


## training

In [11]:
def random_feature_selection(x, t_d=3776, d=2048):
    idx = torch.tensor(sample(range(0, t_d), d)) # t_d(特徴量)の中からランダムにd個サンプリングする(ぜんぶやると重いから) # わからん
    # 0~t_d idxの中からランダムにd個サンプリングする
    x = np.take(x, idx, 1)
    return x

In [12]:
# ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_train_features.pkl')
# # 938(subseq length) * 600(n_samples)
# wav_name = []
# for i in range(len(ext_data['wav_name'])):
#     per_wav_name = [ext_data['wav_name'][i]]
#     per_n_samples = ext_data['features'][0].shape[0]
#     wav_name.extend(per_wav_name * per_n_samples)
# ext_data['wav_name'] = wav_name
# ext_data['features'] = np.concatenate(ext_data['features'], axis=0)

In [13]:
#ext_data['label']

In [14]:
#len(ext_data['wav_name'])

In [15]:
#ext_data['features'].shape

In [16]:
#ext_data['train'].keys()

In [17]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_train_features.pkl')
    # 938(subseq length) * 600(n_samples)
    wav_name = []
    for i in range(len(ext_data['wav_name'])):
        per_wav_name = [ext_data['wav_name'][i]]
        per_n_samples = ext_data['features'][0].shape[0]
        wav_name.extend(per_wav_name * per_n_samples)
    ext_data['wav_name'] = wav_name
    ext_data['features'] = np.concatenate(ext_data['features'], axis=0)
    
    # dataloader
    #for machine in machine_types:
    #    if machine_type == machine:
    #        ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine}_features.pkl')
    #    else:
    #        ext_data_ = pd.read_pickle(f'{INPUT_ROOT}/{machine}_features.pkl')
    #        ext_data['train']['features'] = np.concatenate([ext_data['train']['features'],
    #                                                        ext_data_['train']['features']], axis=0)
    #        ext_data['train']['labels'] = np.concatenate([ext_data['train']['labels'],
    #                                                     ext_data_['train']['labels']], axis=0)
    #        ext_data['train']['wav_names'] = np.concatenate([ext_data['train']['wav_names'],
    #                                                         ext_data_['train']['wav_names']], axis=0)
            
    #t_d = 3776    # 全特徴量数
    #d = 1024      # 使う特徴量数
    #for phase in ['train', 'valid_source', 'valid_target']:
    #    x = ext_data[phase]['features']
    #    x = random_feature_selection(x)
    #    ext_data[phase]['features'] = x
    #    x = ext_data[phase]['features'][:,512:2048]
    #    ext_data[phase]['features'] = x
    dataloaders_dict = modeler.make_dataloader(ext_data)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    in_features = ext_data['features'].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    num_classes = 6
    net = Model(in_features, mid_size, latent_size, num_classes)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['param']['num_epochs']
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
    #                                          max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, num_epochs, writer, model_out_path, score_out_path, pred_out_path, ext_data)
    
    com.toc()

In [18]:
machine_types

['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']

In [19]:
#for machine_type in machine_types:
#    run(machine_type)

In [20]:
#run(machine_types[2])

In [21]:
log_file = config['IO_OPTION']['OUTPUT_ROOT']+'/inference_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_file, 'inference')

In [22]:
#ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_valid_source_features.pkl')

In [23]:
#############################################################################
# run
#############################################################################
def run_inference(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    ext_data = {}
    ext_data['valid_source'] = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_valid_source_features.pkl')
    ext_data['valid_source']['type'] = com.get_section_types(ext_data['valid_source']['wav_name'])
    
    ext_data['valid_target'] = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_valid_target_features.pkl')
    ext_data['valid_target']['type'] = com.get_section_types(ext_data['valid_target']['wav_name'])
    
    
    # define writer for tensorbord
    #os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    #tb_log_dir = TB_DIR + '/' + machine_type
    #writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('INFERENCE')
    # parameter setting
    in_features = ext_data['valid_source']['features'][0].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    num_classes = 6
    net = Model(in_features, mid_size, latent_size, num_classes)
    pretrained_dict = torch.load(model_out_path)
    net.load_state_dict(pretrained_dict, strict=True)
    #optimizer = optim.Adam(net.parameters(), lr=1e-3)
    #num_epochs = config['param']['num_epochs']
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
    #                                          max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.inference_net(net, ext_data, score_out_path, pred_out_path)
    
    com.toc()

In [24]:
for machine_type in machine_types:
    run_inference(machine_type)

2021-05-31 22:50:47,269 - inference - INFO - TARGET MACHINE_TYPE: ToyCar
2021-05-31 22:50:47,270 - inference - INFO - MAKE DATA_LOADER
2021-05-31 22:50:48,984 - inference - INFO - INFERENCE
2021-05-31 22:50:51,868 - pytorch_modeler.py - INFO - valid_source
  0%|          | 0/600 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 600/600 [00:02<00:00, 273.38it/s]
2021-05-31 22:50:54,086 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 600/600 [00:01<00:00, 328.82it/s]


,AUC,pAUC
Source_0,0.430200,0.477895
Source_1,0.440500,0.498947
Source_2,0.457900,0.499474
Target_0,0.509700,0.504737
Target_1,0.466500,0.478947
Target_2,0.518600,0.508421
mean,0.470567,0.494737
h_mean,0.468298,0.494444


2021-05-31 22:50:56,118 - inference - INFO - TARGET MACHINE_TYPE: gearbox
2021-05-31 22:50:56,119 - inference - INFO - MAKE DATA_LOADER


...

100%|██████████| 600/600 [00:01<00:00, 313.11it/s]
2021-05-31 22:53:41,435 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 600/600 [00:01<00:00, 312.12it/s]


,AUC,pAUC
Source_0,0.497800,0.485789
Source_1,0.709200,0.540526
Source_2,0.578100,0.549474
Target_0,0.583600,0.525263
Target_1,0.645000,0.538947
Target_2,0.515600,0.525263
mean,0.588217,0.527544
h_mean,0.579595,0.526706


2021-05-31 22:53:43,398 - inference - INFO - TARGET MACHINE_TYPE: slider
2021-05-31 22:53:43,399 - inference - INFO - MAKE DATA_LOADER


elapsed time: 30.624550581 [sec]


KeyboardInterrupt: 